In [1]:
# load necessary libraries
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import json
import pickle
import glob

from readImages import *
from imageModifiers import *
from readDataFiles import *
from projEdgeBreaks import *
from gaussBreaks import *
from saveImages import *
from plottingFuncs import *

# change pandas parameter
pd.options.mode.chained_assignment = None
# change plot size
mpl.rcParams["figure.figsize"] = (15, 15)

## Read data files
Subject file, classification export, and data from experiment

In [2]:
subjFile = "data/decoding-the-civil-war-subjects-9-29-17.csv"
clFile = 'data/letter-slope-classifications_10nov2017.csv'
experimentFile = "accuracy.pkl"

consensusFolder = "C:/Users/danny/Repos/text_segmentation/consensus/consensus/"
# consensusFile = consensusFolder + "decoding-the-civil-war-consensus-linewise_{mss_label}.csv"
consensusCsvFiles = glob.glob('{}/*.csv'.format(consensusFolder))
consensusCsvFiles = [i for i in consensusCsvFiles if "linewise" in i]

# subject file
subj = readSubjFile(subjFile)
# classification export
clExp = readSlopeClassification(clFile)

# experiment data
with open(experimentFile, "rb") as f:
    data = pickle.load(f)

In [3]:
hdl_id = "mssEC_02_007"
there = np.where(subj["hdl_id"] == hdl_id)[0][0]
url = subj.iloc[there]["url"]
thisn = data[hdl_id]
segments = thisn["segment_results"]
let, grey = readImg(url)

nuOpt = np.arange(0.5, 6.5, 0.5) # 0.5:6.5
biOpt = list(range(1, 10)) # 1:10

In [4]:
nuOpt = [3.5]
biOpt = [2]

In [5]:
# get best one
most = np.inf
mind = (0,0)
for nu in nuOpt:
    for bi in biOpt:
        bi2 = list(segments['biThresh'] == bi)
        nu2 = list(segments['nu'] == nu)
        getm = [bi2[i] and nu2[i] for i in range(len(segments))]
        tmp = segments.loc[getm]
        wrong = np.sum(tmp['n_words_gauss']-2 != tmp['n_words_cons']) * 1.0/ len(tmp)
#         print(wrong, end = " ")
        if wrong < most:
            most = wrong
            mind = (nu, bi)